In [49]:
import pandas as pd
from pathlib import Path
from ics import Calendar, Event
from datetime import datetime, timedelta

In [50]:
# ------------ ΡΥΘΜΙΣΕΙΣ ΧΡΗΣΤΗ ------------
INPUT_EXCEL = "lessons-calendars.xlsm"   # το αρχείο όπου έχεις το sheet Data
INPUT_SHEET = "ExamsJan26"

OUTPUT_EXCEL = "exams_schedule_output.xlsx"
OUTPUT_ICS = "exams_schedule.ics"

DEFAULT_DURATION_HOURS = 2  # αν δεν έχει end_time, θα βάζουμε +2 ώρες
# ------------------------------------------

In [51]:
def week_date_range(any_date):
    # βρίσκουμε τη Δευτέρα της εβδομάδας
    monday = any_date - timedelta(days=any_date.weekday())
    friday = monday + timedelta(days=4)
    return monday, friday

In [52]:
def load_data():
    """Διαβάζει τα δεδομένα από το Excel (sheet Data)."""
    df = pd.read_excel(INPUT_EXCEL, sheet_name=INPUT_SHEET)

    # Βεβαιώσου ότι τα ονόματα στηλών ταιριάζουν με αυτά
    required_cols = [
        "course_id",
        "course_name",
        "semester",
        "instructor",
        "exam_date",
        "start_time",
        "room",
        "notes",
    ]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Λείπουν οι στήλες: {missing}")

    # Μετατροπές τύπων
    df["exam_date"] = pd.to_datetime(df["exam_date"]).dt.date  # μόνο ημερομηνία
    
    # Drop rows where exam_date is missing
    df = df.dropna(subset=["exam_date"])

    # Αν start_time είναι string τύπου "09:00"
    df["start_time"] = df["start_time"].astype(str)

    # Συνένωση σε datetime για αρχή
    df["start_dt"] = pd.to_datetime(
        df["exam_date"].astype(str) + " " + df["start_time"],
        errors="coerce",
    )

    # Υπολογισμός end_dt με default διάρκεια
    # Στήλη εβδομάδας (για weekly views)
    df["week_number"] = df["start_dt"].dt.isocalendar().week

    return df

In [53]:
def export_excel_views(df: pd.DataFrame):
    with pd.ExcelWriter(
        OUTPUT_EXCEL,
        engine="xlsxwriter",
        datetime_format="yyyy-mm-dd hh:mm",
        date_format="yyyy-mm-dd",
    ) as writer:
        # 1. Πρώτη ύλη
        df.to_excel(writer, sheet_name="Data", index=False)

        # 2. Views ανά εξάμηνο (list μορφή)
        semesters = sorted(df["semester"].dropna().unique())
        for sem in semesters:
            df_sem = df[df["semester"] == sem].copy()
            df_sem = df_sem.sort_values("start_dt")
            sheet_name = f"Sem_{sem}"
            df_sem.to_excel(writer, sheet_name=sheet_name, index=False)

        # 3. By_Week view (όπως πριν, αν θες)
        df_week = df.copy()
        df_week = df_week.sort_values(["start_dt"])
        df_week[
            [
                "semester",
                "course_id",
                "course_name",
                "instructor",
                "start_dt",
                "room",
                "notes",
            ]
        ].to_excel(writer, sheet_name="By_Week", index=False)

        # 4. Calendar-style sheets για κάθε εξάμηνο
        for sem in semesters:
            df_sem = df[df["semester"] == sem].copy()
            if not df_sem.empty:
                create_semester_calendar_sheet(df_sem, writer, sem)


In [54]:
def export_ics(df: pd.DataFrame):
    """Φτιάχνει .ics με όλα τα exams."""
    cal = Calendar()

    # Κράτα μόνο όσες εγγραφές έχουν έγκυρο start_dt
    df_valid = df.dropna(subset=["start_dt"])

    for _, row in df_valid.iterrows():
        e = Event()
        # Τίτλος event
        e.name = f"{row['course_name']} (Εξάμ. {row['semester']})"

        # Περιγραφή
        desc_parts = [
            f"Μάθημα: {row['course_name']}",
            f"Κωδικός: {row['course_id']}",
            f"Εξάμηνο: {row['semester']}",
            f"Διδάσκων: {row['instructor']}",
        ]
        if isinstance(row.get("notes"), str) and row["notes"].strip():
            desc_parts.append(f"Σημειώσεις: {row['notes']}")
        e.description = "\n".join(desc_parts)

        # Χρόνοι
        e.begin = row["start_dt"]
        e.end = row["start_dt"] + timedelta(hours=DEFAULT_DURATION_HOURS)

        # Τοποθεσία
        if isinstance(row.get("room"), str) and row["room"].strip():
            e.location = row["room"]

        cal.events.add(e)

    Path(OUTPUT_ICS).write_text(str(cal), encoding="utf-8")
    print(f"✅ Έγινε export σε ICS: {OUTPUT_ICS}")


# def main():
#     df = load_data()
#     export_excel_views(df)
#     export_ics(df)


# if __name__ == "__main__":
#     main()


In [55]:
df = load_data()
df

,course_id,course_name,semester,instructor,exam_date,start_time,end_time,room,epitirites,notes,start_dt,week_number
0,ΓΕΝ002,Γραμμική Άλγεβρα και Αναλυτική Γεωμετρία,1,Παπαϊωάννου,2025-09-09,18:00:00,20:00:00,301.0,"Μπακάλης, Καπαγιαννίδης",NaN,2025-09-09 18:00:00,37
1,ΓΕΝ001,Απειροστικός Λογισμός Ι,1,Παπαϊωάννου,2025-09-05,18:00:00,20:00:00,206.0,"Παπαϊωάννου, Μιχαηλίδης",NaN,2025-09-05 18:00:00,36
2,ΣΥΓ001,Γεωδαισία Ι,1,Καλαμάκης,2025-09-17,12:00:00,14:00:00,NaN,"Τσιαράπας, Κουροπαλάτης",NaN,2025-09-17 12:00:00,38
3,ΓΕΝ003,Φυσική για Μηχανικούς,1,Βοζίκης,2025-09-16,12:00:00,14:00:00,NaN,NaN,NaN,2025-09-16 12:00:00,38
4,ΔΟΜ001,Τεχνικό Σχέδιο,1,Βλαχονάσιου,2025-09-11,09:30:00,11:30:00,NaN,NaN,NaN,2025-09-11 09:30:00,37
...,...,...,...,...,...,...,...,...,...,...,...,...
93,ΣΥΓ018,Σχεδιασμός και Λειτουργία Θαλάσσιων Συστημάτων,9,Γαλάνης,2025-09-24,15:00:00,17:00:00,NaN,NaN,NaN,2025-09-24 15:00:00,39
94,ΣΥΓ019,Σχεδιασμός και Λειτουργία Αεροπορικών Συστημάτων,9,Γαλάνης,2025-09-23,18:00:00,20:00:00,NaN,NaN,NaN,2025-09-23 18:00:00,39
97,ΥΔΡ014,Υδραυλικές Κατασκευές & Φράγματα,9,Κόκκινος,2025-09-23,18:00:00,20:00:00,NaN,NaN,NaN,2025-09-23 18:00:00,39
98,ΥΔΡ015,Εγγειοβελτιωτικά Έργα – Αρδεύσεις,9,Τσιαράπας,2025-09-10,18:00:00,20:00:00,NaN,NaN,NaN,2025-09-10 18:00:00,37


In [56]:
def create_semester_calendar_sheet(df_sem: pd.DataFrame, writer, sem: int):
    """
    Δημιουργεί ένα sheet τύπου calendar για ένα εξάμηνο.
    - στήλες: Monday–Sunday
    - γραμμές: μοναδικές ώρες έναρξης εξετάσεων
    - blocks ανά εβδομάδα (ISO week)
    """
    workbook = writer.book
    sheet_name = f"Sem_{sem}_Calendar"
    worksheet = workbook.add_worksheet(sheet_name)
    writer.sheets[sheet_name] = worksheet

    # Μορφοποιήσεις (προαιρετικά, αλλά βοηθούν)
    header_fmt = workbook.add_format({"bold": True, "align": "center"})
    time_fmt = workbook.add_format({"align": "right"})
    exam_fmt = workbook.add_format({"text_wrap": True, "valign": "top"})

    # Ημέρες εβδομάδας (αγγλικά από pandas, αλλά μπορείς να τα γράψεις ελληνικά)
    days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] #, "Saturday", "Sunday"]
    days_gr = {
        "Monday": "Δευτέρα",
        "Tuesday": "Τρίτη",
        "Wednesday": "Τετάρτη",
        "Thursday": "Πέμπτη",
        "Friday": "Παρασκευή",
        # "Saturday": "Σάββατο",
        # "Sunday": "Κυριακή",
    }

    # Βρίσκουμε τις εβδομάδες που υπάρχουν εξετάσεις
    df_sem = df_sem.copy()
    df_sem["week_number"] = df_sem["start_dt"].dt.isocalendar().week
    df_sem["weekday_name"] = df_sem["start_dt"].dt.day_name()

    weeks = sorted(df_sem["week_number"].unique())

    # Ώρες που θα χρησιμοποιήσουμε ως "γραμμές"
    # Παίρνω όλες τις μοναδικές ώρες έναρξης (με λεπτά) για το εξάμηνο
    df_sem["time_str"] = df_sem["start_dt"].dt.strftime("%H:%M")
    time_slots = sorted(df_sem["time_str"].unique())

    # Λίγη μορφοποίηση στηλών
    worksheet.set_column(0, 0, 10)  # στήλη ώρας
    worksheet.set_column(1, 7, 25)  # στήλες ημερών

    current_row = 0

    for week in weeks:
        # Header για εβδομάδα
        df_week = df_sem[df_sem["week_number"] == week]
        week_monday, week_friday = week_date_range(df_week["start_dt"].min().date())

        title = (
            f"Εξάμ. {sem} - Εβδομάδα "
            f"{week_monday.strftime('%d/%m/%Y')} – {week_friday.strftime('%d/%m/%Y')}"
        )
        worksheet.write(current_row, 0, title, header_fmt)
        current_row += 1

        # Γράφουμε κεφαλίδες ημερών
        worksheet.write(current_row, 0, "")  # κενή πάνω από τις ώρες
        for col, day_en in enumerate(days_order, start=1):
            worksheet.write(current_row, col, days_gr[day_en], header_fmt)
        current_row += 1

        # Filter exams για τη συγκεκριμένη εβδομάδα
        df_week = df_sem[df_sem["week_number"] == week]

        # Για κάθε time slot δημιουργούμε μια γραμμή
        for t in time_slots:
            worksheet.write(current_row, 0, t, time_fmt)

            # Για κάθε μέρα της εβδομάδας, βάζουμε όσα exams ταιριάζουν
            for col, day_en in enumerate(days_order, start=1):
                exams_here = df_week[
                    (df_week["time_str"] == t) &
                    (df_week["weekday_name"] == day_en)
                ]

                if not exams_here.empty:
                    # Αν για κάποιο λόγο έχεις >1 εξέταση την ίδια ώρα/μέρα,
                    # θα τις γράψουμε όλες στο ίδιο κελί, γραμμή-γραμμή.
                    lines = []
                    for _, row in exams_here.iterrows():
                        line = row["course_name"]
                        # Προσθέτουμε ώρα έναρξης & αίθουσα
                        line += f"\n{row['start_dt'].strftime('%H:%M')}"
                        if isinstance(row.get("room"), str) and row["room"].strip():
                            line += f"\nΑίθ.: {row['room']}"
                        if isinstance(row.get("instructor"), str) and row["instructor"].strip():
                            line += f"\nΔιδάσκων: {row['instructor']}"
                        lines.append(line)

                    cell_text = "\n\n".join(lines)
                    worksheet.write(current_row, col, cell_text, exam_fmt)

            current_row += 1

        # Κενή γραμμή ανάμεσα σε εβδομάδες
        current_row += 2

In [57]:
export_excel_views(df)

In [58]:
export_ics(df)

✅ Έγινε export σε ICS: exams_schedule.ics


d:\mypython_projects\civil_ihu_pyappz\.venv\Lib\site-packages\ics\component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(
